<a href="https://colab.research.google.com/github/123-code/Deep-Learning-From-Scratch/blob/main/Micrograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install micrograd


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from micrograd.engine import Value

class Value:
  def __init__(self,data,_children=(),_op="",label=""):
    self.data = data
    self.grad = 0
    self.prev = set(_children)
    self._op = _op
    self.label = label
  
  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self,other):
    result = Value(self.data + other.data,(self,other),"+")
    return result

  def __mul__(self,other):
    result = Value(self.data * other.data,(self,other),"*")
    return result

a = Value(2.0,label="a")
b =  Value(-3.0,label="b")
c = Value(10.0,label="c")
e = a*b; e.label="e"
#a + b
d = e + c; d.label="d"
f = Value(-2.0,label="f")
l = d+f
l
# variable grad should store the gradient of the loss with respect to each value. 



Value(data=2.0)

In [16]:
L = d*f
f.grad = 4.0
d.grad = -2


In [15]:
l.grad = 1

In [ ]:
#Dl/dc
c.grad = -2.0
e.grad = -2.0

Example of gradient check: Derivative of the loss against all the previous multiplication results.

In [24]:
def lol():
  h=0.001
  a = Value(2.0,label="a")
  b =  Value(-3.0,label="b")
  c = Value(10.0,label="c")
  e = a*b; e.label="e"
#a + b
  d = e + c; d.label="d"
  f = Value(-2.0,label="f")
  l = d*f
  l1 = l.data


  
  a = Value(2.0 ,label="a")
  b =  Value(-3.0,label="b")
  c = Value(10.0,label="c")
  e = a*b; e.label="e"
  e.data += h
#a + b
  d = e + c; d.label="d"
  f = Value(-2.0 ,label="f")
  l = d*f
  l2 = l.data
  print((l2-l1)/h)

lol()


-2.000000000000668
